# $L^2_\mu$ training of DeepONet

In [1]:
# MIT License
# Copyright (c) 2025
#
# This is part of the dino_tutorial package
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND.
# For additional questions contact Thomas O'Leary-Roseberry

import os, sys
import torch
from torch import nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm

sys.path.append('../../')

from dinotorch_lite import *


## Load the Data

In [2]:
data_dir = 'data/full_state/'

mq_data_dict = np.load(data_dir+'mq_data.npz')

q_data = mq_data_dict['q_data']
m_data = mq_data_dict['m_data']


# fno_metadata = np.load(data_dir+'fno_metadata.npz')

# d2v = fno_metadata['d2v_param']
# v2d = fno_metadata['v2d_param']
# nx = fno_metadata['nx']
# ny = fno_metadata['ny']

n_data, dQ = q_data.shape
n_data, dM = m_data.shape

print('dQ = ',dQ,', dM = ',dM)

m_train = torch.Tensor(m_data[:-800])
q_train = torch.Tensor(q_data[:-800])

m_test = torch.Tensor(m_data[-200:])
q_test = torch.Tensor(q_data[-200:])



# Set up datasets and loaders
l2train = L2Dataset(m_train,q_train)
l2test = L2Dataset(m_test,q_test)
batch_size = 32

train_loader = DataLoader(l2train, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(l2test, batch_size=batch_size, shuffle=True)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

dQ =  1722 , dM =  861


## $L^2_\mu$ training

In [3]:
# model_settings = fno2d_settings(modes1=4, modes2=4, width=64, n_layers=4, d_out=2)
# model = VectorFNO2D(v2d=[d2v, d2v], d2v=[v2d, v2d], nx=nx, ny=ny, dim=2, settings=model_settings).to(device) 

rQ = 100

model = DeepONetNodal(dM,dQ,rQ)
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

n_epochs = 100

loss_func = normalized_f_mse
# from scipy.sparse import csr_matrix, save_npz, load_npz
# M_output = load_npz(data_dir+'M_output_csr.npz')
# M_torch = scipy_csr_to_torch_csr(M_output).to(torch.float32) 
# M_torch.to(device)
# loss_func = weighted_l2_norm(M_torch)

lr_scheduler = None

optimizer = torch.optim.Adam(model.parameters())

network, history = l2_training(model,loss_func,train_loader, validation_loader,\
                     optimizer,lr_scheduler=lr_scheduler,n_epochs = n_epochs,verbose = True)

rel_error = evaluate_l2_error(model,validation_loader,error_func = loss_func)

print('L2 relative error = ', rel_error)

torch.save(model.state_dict(), data_dir+'l2_model_fno.pth')

616205
epoch =  0
Epoch 1/100, Train Loss: 7.763365e-01
Epoch 1/100, Validation Loss: 5.469161e-01
epoch =  20
Epoch 21/100, Train Loss: 2.959793e-03
Epoch 21/100, Validation Loss: 6.189477e-03
epoch =  40
Epoch 41/100, Train Loss: 1.729799e-03
Epoch 41/100, Validation Loss: 5.397679e-03
epoch =  60
Epoch 61/100, Train Loss: 1.082460e-03
Epoch 61/100, Validation Loss: 4.632592e-03
epoch =  80
Epoch 81/100, Train Loss: 7.637260e-04
Epoch 81/100, Validation Loss: 5.296993e-03
L2 relative error =  0.06498350238635509
